# DATA ANONYMIZATION

In [1]:
# import required libraries
import pandas as pd

## Section 1: Setup

In [30]:
# Load created `students_data.csv` (fake dataset created by `fake_dataset_creation.py`)
students = pd.read_csv("students_data.csv")

# Set display options to show all columns
pd.set_option('display.max_columns', None)

# Check loading is correct 
students.head()

,First_Name,Last_Name,Age,School,School_Address,School_PC,Parents_salary,Parents_occupation,Weight,Size,Feet_size,Eye_color,Hair_color,Previous_year_grades,Current_year_grades
0,Scott,Barnett,15,School B,Street B 456,23456,8238,Teacher,55.4,137.1,36,Green,Black,6.1,6.9
1,Jake,Davis,17,School A,Street A 123,12345,32791,Teacher,68.9,146.0,37,Brown,Red,7.2,3.8
2,Kyle,York,9,School B,Street B 456,23456,12452,Unemployed,35.3,165.2,42,Green,Black,3.8,5.1
3,Amber,Calhoun,18,School A,Street A 123,12345,56305,Nurse,64.4,155.7,38,Brown,Red,7.9,9.8
4,Mark,Green,14,School D,Street D 101,45678,72368,Doctor,75.6,112.4,39,Blue,Brown,7.3,7.3


# Section 2: Data analyze

In [31]:
# Check columns
students.columns

Index(['First_Name', 'Last_Name', 'Age', 'School', 'School_Address',
       'School_PC', 'Parents_salary', 'Parents_occupation', 'Weight', 'Size',
       'Feet_size', 'Eye_color', 'Hair_color', 'Previous_year_grades',
       'Current_year_grades'],
      dtype='object')

In the first instance, 'Eye_color', 'Hair_color', 'First_Name' and 'Last_Name' are not dispensable for the analysis so we remove from the dataset.
We use row ID to identify each student.

In [32]:
# remove 'Eye_colors', 'Hair_color', 'First_Name' and 'Last_Name'  columns
students.drop(['Eye_color', 'Hair_color', 'First_Name' , 'Last_Name'] , axis=1, inplace=True)

# add row id as student identifier
students["ID"] = students.index

# Validate result
students.head()

,Age,School,School_Address,School_PC,Parents_salary,Parents_occupation,Weight,Size,Feet_size,Previous_year_grades,Current_year_grades,ID
0,15,School B,Street B 456,23456,8238,Teacher,55.4,137.1,36,6.1,6.9,0
1,17,School A,Street A 123,12345,32791,Teacher,68.9,146.0,37,7.2,3.8,1
2,9,School B,Street B 456,23456,12452,Unemployed,35.3,165.2,42,3.8,5.1,2
3,18,School A,Street A 123,12345,56305,Nurse,64.4,155.7,38,7.9,9.8,3
4,14,School D,Street D 101,45678,72368,Doctor,75.6,112.4,39,7.3,7.3,4


The salary of the parents is also not relevant for the analysis, the same can be said for the occupation data but in order not to lose all the parents information we will identify with a “Y” or “N” if the parents work or not.

In [33]:
# create a function that returns Y or N depending of the occupacy
def has_occupacy(occupacy):
    if occupacy == "Unemployed":
        return "N"
    return "Y"

# create new column applying the function
students["Parents_employed"] = students["Parents_occupation"].apply(has_occupacy)

# remove `Parents_salary` and `Parents_occupation` data
students.drop(['Parents_salary' , 'Parents_occupation'] , axis=1, inplace=True)

# validate changets
students.head()

,Age,School,School_Address,School_PC,Weight,Size,Feet_size,Previous_year_grades,Current_year_grades,ID,Parents_employed
0,15,School B,Street B 456,23456,55.4,137.1,36,6.1,6.9,0,Y
1,17,School A,Street A 123,12345,68.9,146.0,37,7.2,3.8,1,Y
2,9,School B,Street B 456,23456,35.3,165.2,42,3.8,5.1,2,N
3,18,School A,Street A 123,12345,64.4,155.7,38,7.9,9.8,3,Y
4,14,School D,Street D 101,45678,75.6,112.4,39,7.3,7.3,4,Y


As the objective of the analysis is to determine whether or not the new product improves academic performance we will create a new column that indicates with 0 or 1 whether or not there is performance according to the columns 'Previous_year_grades' and 'Current_year_grades'.

In [34]:
# create a function that returns whether is a performance or not
def performance(prev, actual):
    if actual > prev:
        return 1
    return 0

# apply function to the dataset
students["Performance"] = students.apply(lambda row: performance(row['Previous_year_grades'], row['Current_year_grades']), axis=1)

# remove notes columns
students.drop(['Previous_year_grades','Current_year_grades'] , axis=1, inplace=True)

# check the result
students.head()

,Age,School,School_Address,School_PC,Weight,Size,Feet_size,ID,Parents_employed,Performance
0,15,School B,Street B 456,23456,55.4,137.1,36,0,Y,1
1,17,School A,Street A 123,12345,68.9,146.0,37,1,Y,0
2,9,School B,Street B 456,23456,35.3,165.2,42,2,N,1
3,18,School A,Street A 123,12345,64.4,155.7,38,3,Y,1
4,14,School D,Street D 101,45678,75.6,112.4,39,4,Y,0


We proceed to analyze the school data. The `address' and 'zip code' are very sensitive data and can easily be used to identify students. But as we create the dataset randomly lets chek if exist real evidences that puts in risk student integrity.

Exist relation between zip_code and employment.
Exist relation between age and school.

In [35]:
# remove school sensible data
students.drop(['School_Address','School_PC'] , axis=1, inplace=True)

# check the result
students.head()

,Age,School,Weight,Size,Feet_size,ID,Parents_employed,Performance
0,15,School B,55.4,137.1,36,0,Y,1
1,17,School A,68.9,146.0,37,1,Y,0
2,9,School B,35.3,165.2,42,2,N,1
3,18,School A,64.4,155.7,38,3,Y,1
4,14,School D,75.6,112.4,39,4,Y,0


To determine whether 'School' column must be erased or not we will see if the school is directly related with parent employment and studens age. 

Normally the age is directly related to the school, as we create the dataset randomly lets check if has this relationship.

Etapa infantil: de 0 a 6 años. Se imparte en las “escuelas infantiles”.

Educación Primaria: de 6 a 12 años. Se imparte en “colegios”.

Educación Secundaria Obligatoria (ESO): de 12 a 16 años. Los centros públicos que la imparten se llaman “institutos” o también “IES”. Si el centro es concertado o privado, se le llama "colegio".

Bachillerato: de 16 a 18 años. En institutos o IES. Si el centro es concertado o privado, se le llama "colegio".

Formación Profesional (FP): de 16 años (o 15 excepcionalmente) en adelante. En institutos, IES o centros de FP.

Same for parents employment.

In [ ]:
# check age and school relation. If exist a school with only an age value remove directly

# check parents employment and school relation. If exist a direct relationship remove directly.

Cada rango de edad pertenece a un curso. Cambia columna age a esta logica.